In [1]:
import os
from src.pix2pix_turbo import Pix2Pix_Turbo
from PIL import Image
import torch
from torchvision import transforms
import torchvision.transforms.functional as F

In [2]:
models = {
    0: 'output/pix2pix_turbo/angel_0_go/checkpoints/model_1001.pkl', # 180.44
    1: 'output/pix2pix_turbo/angel_1/checkpoints/model_501.pkl', # 211.78
    2: 'output/pix2pix_turbo/angel_2/checkpoints/model_501.pkl', # 230.76
    3: 'output/pix2pix_turbo/backup/angel_3/checkpoints/model_501.pkl', # 228.69 
    4: 'output/pix2pix_turbo/angel_4/checkpoints/model_4501.pkl', # 187.45
    5: 'output/pix2pix_turbo/angel_5/checkpoints/model_3001.pkl' # 173.64
}

In [3]:
src_dir = 'data/oct/validation_set/val/CF/'
dst_dir = 'data/oct/generated_OCT_images_DIR'

In [4]:
def generate_images(model_index, src_dir, dst_dir):
    model = Pix2Pix_Turbo(pretrained_path=models[model_index])
    model.set_eval()
    #model.half()

    for _, dirs, files in os.walk(src_dir):
        
        for f in files:
            f_path = os.path.join(src_dir, f)
            input_image = Image.open(f_path).convert('RGB')
            new_width = input_image.width - input_image.width % 8
            new_height = input_image.height - input_image.height % 8
            input_image = input_image.resize((512, 512), Image.LANCZOS)
            bname = os.path.basename(f)

            # translate the image
            with torch.no_grad():
                c_t = F.to_tensor(input_image).unsqueeze(0).cuda()
                #c_t = c_t.half()
                if 'L' in f:
                    test_prompts = "This is the color fundus image of left eye, please generate corresponding optical coherence tomography(OCT) image"
                else:
                    test_prompts = "This is the color fundus image of right eye, please generate corresponding optical coherence tomography(OCT) image"

                output_image = model(c_t, test_prompts)
                output_pil = transforms.ToPILImage()(output_image[0].cpu() * 0.5 + 0.5)
                
            dst_dir_path = os.path.join(dst_dir, bname.split('.')[0])
            if not os.path.exists(dst_dir_path):
                os.makedirs(dst_dir_path)
            output_pil = output_pil.resize((448, 320), Image.LANCZOS)
            output_pil.save(os.path.join(dst_dir_path, f"{bname.split('.')[0]}_{model_index}.jpg"))
    model.release_memory()

In [5]:
for i in [1, 3]:
    generate_images(i, src_dir, dst_dir)

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
import os
import zipfile

def zip_directory(folder_path, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                # 获取文件的完整路径
                file_path = os.path.join(root, file)
                # 在压缩包中存储的路径，去除父目录路径部分
                arcname = os.path.relpath(file_path, start=folder_path)
                # 添加文件到压缩包中
                zipf.write(file_path, arcname)
    print(f"Folder '{folder_path}' has been compressed into '{output_zip}'")

# 使用示例
zip_directory("data/oct/generated_OCT_images_DIR", "submit_data.zip")


Folder 'data/oct/generated_OCT_images_DIR' has been compressed into 'submit_data.zip'
